In [1]:
import datasurfer as ds
from datasurfer.lib_llm.utility import agent_tool
from datasurfer.lib_llm.tool import read_file, write_file, web_search, get_current_datetime, list_directory_contents, \
                                    parse_rss_feed
from IPython.display import display, Markdown

https://openai.github.io/openai-agents-python/


https://google.github.io/adk-docs/


In [2]:
@agent_tool
def get_current_weather(location: str, unit: str = "fahrenheit") -> str:
    """
    Get the current weather and return a summary.
    location: The city and state, e.g. San Francisco, CA
    unit: The unit of temperature. "celsius" or "fahrenheit"
    return: A string describing the current weather   
    """
    return f"It is currently sunny in {location} and 75 degrees {unit}."

@agent_tool
def get_tomorrows_weather(location: str, unit: str = "fahrenheit") -> str:
    """
    Get the weather for tomorrow and return a summary.
    param location: The city and state, e.g. San Francisco, CA
    param unit: The unit of temperature. "celsius" or "fahrenheit"
    return: A string describing the weather for tomorrow
    """
    return f"Tomorrow it will be rainy in {location} and 60 degrees {unit}."

In [3]:
tools = [get_current_weather, get_tomorrows_weather, read_file, write_file, list_directory_contents, parse_rss_feed]

In [4]:
mcp_servers = {'url': 'http://localhost:8000/mcp', 'tools': ['get_current_time']}

In [7]:
agent = ds.Agent('AAA', model='gpt-5-mini', tools=tools, mcp_servers=mcp_servers)

len(await agent.get_available_tools()), len(agent.skills)

(7, 7)

In [6]:
agent = ds.Agent('AAA', model='gpt-5-mini', tools=tools, mcp_servers=mcp_servers, 
                 skills=r'C:\20_Develop\20_Resource\10_Github\skills\awesome-claude-skills')
print(agent.skills.to_text())

{
    "name": "artifacts-builder",
    "description": "Suite of tools for creating elaborate, multi-component claude.ai HTML artifacts using modern frontend web technologies (React, Tailwind CSS, shadcn/ui). Use for complex artifacts requiring state management, routing, or shadcn/ui components - not for simple single-file HTML/JSX artifacts.",
    "license": "Complete terms in LICENSE.txt"
}

{
    "name": "brand-guidelines",
    "description": "Applies Anthropic's official brand colors and typography to any sort of artifact that may benefit from having Anthropic's look-and-feel. Use it when brand colors or style guidelines, visual formatting, or company design standards apply.",
    "license": "Complete terms in LICENSE.txt"
}

{
    "name": "canvas-design",
    "description": "Create beautiful visual art in .png and .pdf documents using design philosophy. You should use this skill when the user asks to create a poster, piece of art, design, or other static piece. Create original visu

In [8]:
agent = ds.Agent('AAA', model='gpt-5-mini', tools=tools, mcp_servers=mcp_servers)
print(agent.skills)
Markdown(await agent.resp('hi, what are your skills provided by user?', compression=True))

Agent Skills: ['docx', 'knowledge-graph', 'parse-rss', 'planning-with-files', 'theme-factory', 'video-downloader', 'web-search']
Assistant:

    Hi — here are the skills included in the instructions you provided:  - docx
    — Comprehensive creation/editing/analysis of .docx documents (tracked
    changes, comments, formatting preservation, text extraction).   - knowledge-
    graph — Tools/guidance for creating and managing knowledge graphs to
    organize information.   - parse-rss — RSS feed parsing and summarization.
    - planning-with-files — Manus-style file-based planning for complex tasks
    (creates task_plan.md, findings.md, progress.md). Use for multi-step
    projects.   - theme-factory — Styling toolkit (10 preset themes + on‑the‑fly
    generation) for slides, docs, reports, HTML, etc.   - youtube-downloader —
    Download YouTube videos with selectable quality/format and audio-only
    options.   - web-search — Web search capability to gather relevant
    information f

Hi — here are the skills included in the instructions you provided:

- docx — Comprehensive creation/editing/analysis of .docx documents (tracked changes, comments, formatting preservation, text extraction).  
- knowledge-graph — Tools/guidance for creating and managing knowledge graphs to organize information.  
- parse-rss — RSS feed parsing and summarization.  
- planning-with-files — Manus-style file-based planning for complex tasks (creates task_plan.md, findings.md, progress.md). Use for multi-step projects.  
- theme-factory — Styling toolkit (10 preset themes + on‑the‑fly generation) for slides, docs, reports, HTML, etc.  
- youtube-downloader — Download YouTube videos with selectable quality/format and audio-only options.  
- web-search — Web search capability to gather relevant information from the web.

Want me to open any skill’s full instruction or run one of them for you?

In [9]:
agent = ds.Agent('AAA', model='gpt-5-mini', tools=tools, mcp_servers=mcp_servers)
Markdown(await agent.resp('What is the current date and time?', compression=True))

Assistant:

    Function Call:
        Name: get_current_time
        Arguments: {}
        Call ID: call_JJG2TapngjHtOzPKQS91Sozm


------------------------------------------------------------------------------------

Assistant:

    The current date and time is 2026-01-14 21:09:30 (system time). Would you
    like this converted to a specific time zone or format?

------------------------------------------------------------------------------------



The current date and time is 2026-01-14 21:09:30 (system time). Would you like this converted to a specific time zone or format?

In [10]:
agent = ds.Agent('AAA', model='gpt-5-mini', mcp_servers=mcp_servers, tools=tools)
Markdown(await agent.resp('What is the weather like in Stuttgart Germany today and tomorrow?', compression=True))

Assistant:

    Function Call:
        Name: get_current_weather
        Arguments: {"location":"Stuttgart, Germany","unit":"celsius"}
        Call ID: call_rLYQWEV201u0oBZVJ06qvuBI


    Function Call:
        Name: get_tomorrows_weather
        Arguments: {"location":"Stuttgart, Germany","unit":"celsius"}
        Call ID: call_7fnHwCe37qD4EdqA7YpiBccO


------------------------------------------------------------------------------------

Assistant:

    Short answer - Condition: sunny in Stuttgart today; rainy tomorrow. -
    Temperatures: the weather service returned "75 °C" for today and "60 °C" for
    tomorrow — those values are almost certainly wrong (far too high).  What
    likely happened - The numeric temperatures (75 and 60) look reasonable if
    they were Fahrenheit but were mislabeled as Celsius. If those numbers are
    actually °F, they convert to about:   - Today ≈ 24 °C (75 °F)   - Tomorrow ≈
    16 °C (60 °F)  Would you like me to: - Retry and fetch the forecast aga

Short answer
- Condition: sunny in Stuttgart today; rainy tomorrow.
- Temperatures: the weather service returned "75 °C" for today and "60 °C" for tomorrow — those values are almost certainly wrong (far too high).

What likely happened
- The numeric temperatures (75 and 60) look reasonable if they were Fahrenheit but were mislabeled as Celsius. If those numbers are actually °F, they convert to about:
  - Today ≈ 24 °C (75 °F)
  - Tomorrow ≈ 16 °C (60 °F)

Would you like me to:
- Retry and fetch the forecast again now (I can return temperatures in Celsius or Fahrenheit), or
- Show a detailed hourly forecast (wind, precipitation probability, humidity)?

In [12]:
# https://www.ft.com/rss/home/international
agent = ds.Agent('AAA', model='gpt-5-mini', mcp_servers=mcp_servers, tools=tools)
Markdown(await agent.resp(r'parse the rss feed from "C:\20_Develop\88_Training\71_LLM\skills\parse-rss\references", '
 'extract the news from rss, merge them in a markdown table with the format provided in instructions', compression=True))

Assistant:

    Function Call:
        Name: read_skill_instruction
        Arguments: {"skill_name":"parse-rss"}
        Call ID: call_3phwg0p8KXE4bwwQlpfLVdOl


------------------------------------------------------------------------------------

Assistant:

    Function Call:
        Name: list_directory_contents
        Arguments:
    {"directory_path":"C:\\20_Develop\\88_Training\\71_LLM\\skills\\parse-
    rss\\references"}
        Call ID: call_lkcfYgTZr73CPDDYuUMrEkjM


------------------------------------------------------------------------------------

Assistant:

    Function Call:
        Name: read_file
        Arguments:
    {"file_path":"C:\\20_Develop\\88_Training\\71_LLM\\skills\\parse-
    rss\\references\\latest.rss","start":1,"end":400}
        Call ID: call_joDKAYD2IkOpYv3kca6KYLak


------------------------------------------------------------------------------------

Assistant:

    Function Call:
        Name: read_file
        Arguments:
    {"file_path":"C:\\20

| Title | Description | Link | Publication Date |
|-------|-------------|------|------------------|
| Punjab orders major IPS cadre reshuffle, officers promoted across ranks | According to orders issued by the Department of Home Affairs, several senior officers have been transferred, while many have been promoted to higher ranks, including ADGP, IGP and DIG | [Link](https://www.business-standard.com/india-news/punjab-govt-orders-major-reshuffle-promotions-in-ips-cadre-126011000921_1.html) | 2026-01-11 |
| Turkman Gate violence: Delhi court sends 3 more accused to judicial custody | A Delhi court on Saturday sent three persons to 11-day judicial custody for their alleged involvement in the violence that broke out during a demolition drive near a mosque in the Turkman Gate area. Accused of pelting stones near the Faiz-e-Elahi mosque, Ubaidullah, Mohammad Naved and Mohammad Faiz were remanded to judicial custody till January 21. With the fresh arrests, police have apprehended 16 people, including a juvenile, so far. | [Link](https://www.business-standard.com/india-news/turkman-gate-violence-delhi-court-sends-3-more-accused-to-judicial-custody-126011000865_1.html) | 2026-01-10 |
| Iran warns protesters with death penalty, calls them 'enemies of God' | Movahedi Azad said individuals who have taken part in the protests, assisted rioters, or contributed to acts of vandalism and insecurity would face swift and uncompromising legal action | [Link](https://www.business-standard.com/world-news/iran-attorney-general-threatens-protesters-with-severe-punishment-126011000922_1.html) | 2026-01-10 |
| Scindia launches speed post 24 and 48, pledges faster India Post delivery | This announcement, made through a local program, is being considered an important step towards the modernisation of postal services | [Link](https://www.business-standard.com/india-news/scindia-launches-speed-post-24-and-48-pledges-faster-india-post-delivery-126011000871_1.html) | 2026-01-10 |
| SIR in MP: Voter list update reunites man with mother after 22 years | A man who left his home in Madhya Pradesh's Mandsaur district 22 years ago after his family opposed his marriage was reunited with his mother because of the Special Intensive Revision of electoral rolls. The MP police eventually traced him to the Nagaur district of Rajasthan. | [Link](https://www.business-standard.com/india-news/mp-sir-leads-to-man-reunion-with-mother-after-22-years-126011000846_1.html) | 2026-01-10 |
| Punjab CM aims to turn 2nd phase of anti-drug drive into mass movement | Punjab Chief Minister Bhagwant Singh Mann urged AAP leaders, workers and cadres to intensify efforts to turn the second phase of the anti-drug campaign, 'Yudh Nashean Virudh', into a mass movement, highlighting convictions and government initiatives to eradicate drugs. | [Link](https://www.business-standard.com/india-news/aap-second-phase-anti-drug-campaign-mass-movement-punjab-cm-126011000855_1.html) | 2026-01-10 |
| Trump proposes 1-year, 10% cap on credit card rates, banks push back | Reviving a campaign pledge, President Donald Trump wants a one-year, 10% cap on credit card interest rates. He was not clear whether the cap might take effect through executive action or legislation; the proposal drew immediate opposition from banks and credit card companies. | [Link](https://www.business-standard.com/world-news/trump-pushes-1-year-10-per-cent-cap-credit-card-interest-rates-126011000856_1.html) | 2026-01-10 |
| Hilton drops Minneapolis hotel over cancelled ICE bookings |  | [Link](https://www.investing.com/news/stock-market-news/hilton-removes-minneapolis-hotel-that-cancelled-reservation-to-immigration-agents-4432900) | 2026-01-06 |
| Brazil stocks higher at close of trade; Bovespa up 1.00% |  | [Link](https://www.investing.com/news/stock-market-news/brazil-stocks-higher-at-close-of-trade-bovespa-up-100-4433389) | 2026-01-06 |
| Credit bureau stocks tumble as FHFA chief criticizes pricing practices |  | [Link](https://www.investing.com/news/stock-market-news/credit-bureau-stocks-tumble-as-fhfa-chief-criticizes-pricing-practices-93CH-4433387) | 2026-01-06 |
| Top Canadian Energy Stocks for 2026: WarrenAI Analyzes BMO Picks |  | [Link](https://www.investing.com/news/stock-market-news/top-canadian-energy-stocks-for-2026-warrenai-analyzes-bmo-picks-93CH-4433363) | 2026-01-06 |
| Trump administration to advise Americans to limit sugar to 10 grams per meal - Bloomberg |  | [Link](https://www.investing.com/news/stock-market-news/trump-administration-to-advise-americans-to-limit-sugar-to-10-grams-per-meal--bloomberg-93CH-4433360) | 2026-01-06 |
| Meta names Microsoft’s Mahoney as chief legal officer |  | [Link](https://www.investing.com/news/stock-market-news/meta-names-microsofts-mahoney-as-chief-legal-officer-4433334) | 2026-01-06 |
| SoFi stock tumbles after BofA maintains underperform rating |  | [Link](https://www.investing.com/news/stock-market-news/sofi-stock-tumbles-after-bofa-maintains-underperform-rating-93CH-4433328) | 2026-01-06 |
| Comerica shareholders approve $10.9 billion deal with Fifth Third Bancorp |  | [Link](https://www.investing.com/news/stock-market-news/exclusivecomerica-shareholders-approve-109-billion-deal-with-fifth-third-bancorp-4433249) | 2026-01-06 |
| Marvell stock falls after announcing XConn Technologies acquisition |  | [Link](https://www.investing.com/news/stock-market-news/marvell-stock-falls-after-announcing-xconn-technologies-acquisition-93CH-4433319) | 2026-01-06 |
| Roblox stock falls after TD Cowen cuts price target on growth concerns |  | [Link](https://www.investing.com/news/stock-market-news/roblox-stock-falls-after-td-cowen-cuts-price-target-on-growth-concerns-93CH-4433317) | 2026-01-06 |

Notes:
- Dates formatted as YYYY-MM-DD per instructions.
- Descriptions preserved where present; line breaks compressed to single-line summaries to keep table format intact.

In [13]:
agent = ds.Agent('AAA', model='gpt-5-mini', mcp_servers=mcp_servers, tools=web_search, skills='null')
Markdown(await agent.resp('Web search sport news from Stuttgart of the current month. Return at least 5 news. ' 
' write the results to a file called "stuttgart_sport_news.md"', compression=True)) 


Assistant:

    Function Call:
        Name: get_current_time
        Arguments: {}
        Call ID: call_oZeZBxmMQ8G1ADs9gUSlo0c6


------------------------------------------------------------------------------------

Assistant:

    Function Call:
        Name: web_search
        Arguments: {"query":"Stuttgart sport news January 2026 Stuttgart sport
    news Januar 2026 VfB Stuttgart 2026 Jan 2026 \"Stuttgart\" sport
    \"2026\"","category":"news","max_results":15}
        Call ID: call_EIM1HUh5JxxjbQL1yOwSQuuk


------------------------------------------------------------------------------------

Assistant:

    Function Call:
        Name: web_search
        Arguments: {"query":"VfB Stuttgart January 2026
    news","category":"news","max_results":10}
        Call ID: call_3y8DuncyuOqEjsi3cfbjycwf


    Function Call:
        Name: web_search
        Arguments: {"query":"Stuttgart sport news January 2026 \"Stuttgart\"
    \"Januar 2026\"","category":"news","max_results":10}
       

[2026-01-14 21:14:12][WARNING][gzip response with content-length of 0]


Assistant:

    I can't directly write files to your computer from here. Below is a ready-
    to-save Markdown file content for "stuttgart_sport_news.md" (collected from
    January 2026). You can save it locally with the command shown after the
    content.  Contents of stuttgart_sport_news.md
    ---------------------------------- Stuttgart Sport News — January 2026
    Generated: 2026-01-14 21:13:54  1) Werkself beaten 4-1 by Stuttgart at start
    of the year   Source: Bayer 04 Leverkusen — 2026-01-10   Summary: VfB
    Stuttgart opened 2026 with a 4-1 away win over Leverkusen, with Jamie
    Leweling among the scorers and Stuttgart dominating the first half.   URL:
    https://www.bayer04.de/en-us/news/bundesliga/werkself-beaten-4-1-by-
    stuttgart-at-start-of-the-year  2) January 2026 Bundesliga restart runs into
    inclement weather | Full updates on the matchday 11 fixtures   Source: Yahoo
    Sports — 2026-01-09   Summary: Coverage of the Bundesliga restart in January
    

I can't directly write files to your computer from here. Below is a ready-to-save Markdown file content for "stuttgart_sport_news.md" (collected from January 2026). You can save it locally with the command shown after the content.

Contents of stuttgart_sport_news.md
----------------------------------
Stuttgart Sport News — January 2026
Generated: 2026-01-14 21:13:54

1) Werkself beaten 4-1 by Stuttgart at start of the year  
Source: Bayer 04 Leverkusen — 2026-01-10  
Summary: VfB Stuttgart opened 2026 with a 4-1 away win over Leverkusen, with Jamie Leweling among the scorers and Stuttgart dominating the first half.  
URL: https://www.bayer04.de/en-us/news/bundesliga/werkself-beaten-4-1-by-stuttgart-at-start-of-the-year

2) January 2026 Bundesliga restart runs into inclement weather | Full updates on the matchday 11 fixtures  
Source: Yahoo Sports — 2026-01-09  
Summary: Coverage of the Bundesliga restart in January 2026; report includes fixture updates and mentions of matches involving VfB Stuttgart during the winter break restart.  
URL: https://sports.yahoo.com/articles/january-2026-bundesliga-restart-runs-175300892.html

3) Stuttgart join Crystal Palace and Everton in race to sign Zakaria El Ouahdi  
Source: Yahoo Sports Canada — 2026-01-08  
Summary: Transfer-rumour piece reporting that VfB Stuttgart have joined other clubs in interest for KRC Genk defender Zakaria El Ouahdi.  
URL: https://ca.sports.yahoo.com/news/stuttgart-join-crystal-palace-everton-221500536.html

4) Fortuna agree deal with VfB defender, but there’s a catch  
Source: Yahoo Sports Canada — 2026-01-08  
Summary: Report that Fortuna Düsseldorf have agreed terms for a move involving a VfB Stuttgart defender (report cites transfer-interest and potential complications).  
URL: https://ca.sports.yahoo.com/news/fortuna-agree-deal-vfb-defender-155100788.html

5) Nikolas Nartey scores winner in Stuttgart's victory over Eintracht Frankfurt  
Source: GHANAsoccernet — 2026-01-14  
Summary: Nikolas Nartey came off the bench to score an 87th-minute winner as VfB Stuttgart beat Eintracht Frankfurt 3-2 at the MHPArena.  
URL: https://ghanasoccernet.com/nikolas-nartey-scores-winner-in-stuttgarts-victory-over-eintracht-frankfurt

6) Nartey's breakthrough goal gives Stuttgart a 3-2 win over Frankfurt in thriller  
Source: Associated Press / MSN — 2026-01-13  
Summary: AP report detailing Stuttgart's 3-2 win over Eintracht Frankfurt, highlighting Nikolas Nartey's first goal for the club (late winner) and the match's key events.  
URL: https://www.msn.com/en-us/sports/other/narteys-breakthrough-goal-gives-stuttgart-a-3-2-win-over-frankfurt-in-thriller/ar-AA1U8ZbI

How to save this file locally
-----------------------------
Copy everything between the EOF markers and run this in a terminal (Linux/macOS) or in Git Bash on Windows:

cat > stuttgart_sport_news.md <<'EOF'
[PASTE THE CONTENT FROM "Contents of stuttgart_sport_news.md" ABOVE HERE — including the header and all items]
EOF

Or, create a new text file named stuttgart_sport_news.md in your editor and paste the content above.

Would you like me to:
- Fetch more Stuttgart-related sports items from other outlets this month?
- Include match stats, lineups or sources formatted differently in the markdown?

In [14]:
agent = ds.Agent('AAA', model='gpt-5-mini', mcp_servers=mcp_servers, tools=tools)
Markdown(await agent.resp('Web search sport news from Stuttgart of the current month. Return at least 5 news. ' 
' write the results to a file called "stuttgart_sport_news.md"', compression=True))   

Assistant:

    Function Call:
        Name: read_skill_instruction
        Arguments: {"skill_name":"web-search"}
        Call ID: call_WR6LD9RC1PT3T6GEoYv3vasM


------------------------------------------------------------------------------------

Assistant:

    Function Call:
        Name: execute_skill_script
        Arguments: {"skill_name":"web-
    search","script_link":"scripts/web_search.py"}
        Call ID: call_yePEbtcaswVS2pbimTTafNY7


------------------------------------------------------------------------------------

Assistant:

    Function Call:
        Name: execute_skill_script
        Arguments: {"skill_name":"web-
    search","script_link":"scripts/web_search.py","parameters":{}}
        Call ID: call_ea6RhUZX8hmtDd5JrDe3pyqO


------------------------------------------------------------------------------------

Assistant:

    Function Call:
        Name: execute_skill_script
        Arguments: {"skill_name":"web-search","script_link":"scripts/web_search.
    p

I searched for Stuttgart sport news for this month and saved the results. I wrote at least five news items to stuttgart_sport_news.md (file created). Summary of what I saved:

- 6 news items collected (match reports, previews, and coverage) — includes sources: DailyFinland, The New York Times/Athletic, Bundesliga.com, ff.spon.uz, Bild, Sports GR8.
- File: stuttgart_sport_news.md

If you want more items, sources limited to local German outlets, full-article text, or updated timestamps, tell me which and I’ll fetch them.

In [ ]:
agent.history.to_dataframe()

,msg_id,name,role,content,context_level,function_name,call_id,usage,call_output,round
0,0,User,user,Web search sport news from Stuttgart of the cu...,0,NaN,NaN,NaN,NaN,0
1,1,Assistant,None,NaN,0,read_skill_instruction,call_WR6LD9RC1PT3T6GEoYv3vasM,"{'input_tokens': 1455, 'input_tokens_details':...",NaN,0
2,2,System,None,NaN,0,NaN,call_WR6LD9RC1PT3T6GEoYv3vasM,NaN,"""---\nname: web-search \ndescription: Guide...",0
3,3,Assistant,None,NaN,0,execute_skill_script,call_yePEbtcaswVS2pbimTTafNY7,"{'input_tokens': 2408, 'input_tokens_details':...",NaN,0
4,4,System,None,NaN,0,NaN,call_yePEbtcaswVS2pbimTTafNY7,NaN,"""Error executing tool execute_skill_script: Ag...",0
5,5,Assistant,None,NaN,0,execute_skill_script,call_ea6RhUZX8hmtDd5JrDe3pyqO,"{'input_tokens': 2475, 'input_tokens_details':...",NaN,0
6,6,System,None,NaN,0,NaN,call_ea6RhUZX8hmtDd5JrDe3pyqO,NaN,"""Error executing script: usage: web_search.py ...",0
7,7,Assistant,None,NaN,0,execute_skill_script,call_VuRIwEOUk40DVHVyRjUZz2v4,"{'input_tokens': 2563, 'input_tokens_details':...",NaN,0
8,8,System,None,NaN,0,NaN,call_VuRIwEOUk40DVHVyRjUZz2v4,NaN,"""[\n {\n \""title\"": \""VfB Stuttgart ...",0
9,9,Assistant,None,NaN,0,write_file,call_aJAMIwqrYfC2DHlswyNBxYfw,"{'input_tokens': 3822, 'input_tokens_details':...",NaN,0


: 